# **1. DES (Data Encryption Standard)**

In [1]:
from bitarray import bitarray
from bitarray.util import ba2int, int2ba

In [2]:
def hex_to_bin(hex_string):
    hex_string = hex_string.replace(" ", "")
    return bitarray(bin(int(hex_string, 16))[2:].zfill(len(hex_string) * 4))

In [3]:
def bin_to_hex(bin_array):
    """Chuyển đổi dữ liệu nhị phân thành chuỗi thập lục phân với định dạng mong muốn."""
    hex_string = format(ba2int(bin_array), 'X').zfill(len(bin_array) // 4)
    return ' '.join(hex_string[i:i+2] for i in range(0, len(hex_string), 2))

In [4]:
# message và key dưới dạng hex
hex_message = "01 A2 B3 C4 D5 E6 07 F8"
hex_key = "13 34 57 79 9B BC DF F1"

In [5]:
binary_message = hex_to_bin(hex_message)
binary_key = hex_to_bin(hex_key)
print(f"Dữ liệu nhị phân: {binary_message.to01()}")
print(f"Khóa nhị phân: {binary_key.to01()}")
print(len(binary_message))

Dữ liệu nhị phân: 0000000110100010101100111100010011010101111001100000011111111000
Khóa nhị phân: 0001001100110100010101110111100110011011101111001101111111110001
64


In [6]:
print(f"Dữ liệu thập lục phân: {bin_to_hex(binary_message)}")
print(f"Khóa thập lục phân: {bin_to_hex(binary_key)}")

Dữ liệu thập lục phân: 01 A2 B3 C4 D5 E6 07 F8
Khóa thập lục phân: 13 34 57 79 9B BC DF F1


In [7]:
# read S_blocks
import json
with open("S_blocks.json", "r") as file:
    S_BLOCKS = json.load(file)

# read P_blocks
with open("P_blocks.json", "r") as file:
    P_BLOCKS = json.load(file)

#read PC_blocks
with open("PC_blocks.json", "r") as file:
    PC_BLOCKS = json.load(file)

In [8]:
SHIFT_SCHEDULE = [1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1]

In [9]:
def permute(bits, table):
    """Áp dụng hoán vị theo bảng cho trước."""
    return bitarray([bits[i - 1] for i in table])

In [10]:
def initial_permutation(bits):
    """Áp dụng hoán vị ban đầu."""
    return permute(bits, P_BLOCKS["Initial_Permutation"])

In [11]:
def final_permutation(bits):
    """Áp dụng hoán vị cuối cùng."""
    return permute(bits, P_BLOCKS["Final_Permutation"])

In [12]:
ip_message = initial_permutation(binary_message)
print(f"Initial Permutation: {ip_message.to01()}")

Initial Permutation: 1011100010010100011110000101010110111110101001101000000001100110


In [13]:
ip_message_left = ip_message[:32]
ip_message_right = ip_message[32:]
print(f"Left: {ip_message_left.to01()}")
print(f"Right: {ip_message_right.to01()}")

Left: 10111000100101000111100001010101
Right: 10111110101001101000000001100110


In [14]:
def shift_left(bits, n):
    """Dịch trái n bit."""
    return bits[n:] + bits[:n]

In [15]:
def generate_subkeys(key):
    """Tạo 16 khóa con từ khóa 64-bit ban đầu."""
    key = permute(key, PC_BLOCKS["PC-1"])  # Áp dụng PC-1 (giảm từ 64-bit xuống 56-bit)
    C, D = key[:28], key[28:]  # Chia thành hai phần C và D
    subkeys = []
    for shift in SHIFT_SCHEDULE:
        C, D = shift_left(C, shift), shift_left(D, shift)
        subkeys.append(permute(C + D, PC_BLOCKS["PC-2"]))  # Áp dụng PC-2
    return subkeys

In [16]:
subkeys = generate_subkeys(binary_key)
print(f"Số lượng khóa con: {len(subkeys)}")
# in hết các khóa con
for i, key in enumerate(subkeys):
    print(f"Subkey {i + 1}: {key.to01()}")

Số lượng khóa con: 16
Subkey 1: 000110110000001011101111111111000111000001110010
Subkey 2: 011110011010111011011001110110111100100111100101
Subkey 3: 010101011111110010001010010000101100111110011001
Subkey 4: 011100101010110111010110110110110011010100011101
Subkey 5: 011111001110110000000111111010110101001110101000
Subkey 6: 011000111010010100111110010100000111101100101111
Subkey 7: 111011001000010010110111111101100001100010111100
Subkey 8: 111101111000101000111010110000010011101111111011
Subkey 9: 111000001101101111101011111011011110011110000001
Subkey 10: 101100011111001101000111101110100100011001001111
Subkey 11: 001000010101111111010011110111101101001110000110
Subkey 12: 011101010111000111110101100101000110011111101001
Subkey 13: 100101111100010111010001111110101011101001000001
Subkey 14: 010111110100001110110111111100101110011100111010
Subkey 15: 101111111001000110001101001111010011111100001010
Subkey 16: 110010110011110110001011000011100001011111110101


In [17]:
def f_function(right, key):
    """Hàm f được sử dụng trong mỗi vòng lặp DES."""
    expanded = permute(right, P_BLOCKS["Expansion_Permutation"])
    xor_result = expanded ^ key
    output = bitarray()
    for i in range(8):
        block = xor_result[i * 6:(i + 1) * 6]
        row = int(str(block[0]) + str(block[-1]), 2)
        col = int(block[1:-1].to01(), 2)
        output += int2ba(S_BLOCKS[f'S{i+1}'][row][col], length=4)
    return permute(output, P_BLOCKS["Permutation_Function"])

In [18]:
def DES_round(left, right, key):
    """Một vòng lặp DES."""
    next_left = right
    next_right = left ^ f_function(right, key)
    return next_left, next_right

In [19]:
def cipher_DES(block_64_bit, key, encrypt=True):
    """Mã hóa hoặc giải mã một khối 64-bit bằng DES."""
    block_64_bit = initial_permutation(block_64_bit)
    left, right = block_64_bit[:32], block_64_bit[32:]
    subkeys = generate_subkeys(key) if encrypt else generate_subkeys(key)[::-1]
    
    results = []
    for key in subkeys:
        left, right = DES_round(left, right, key)
        results.append((left.copy(), right.copy(), key.copy()))
    
    final_result = final_permutation(right + left)
    return final_result, results

In [20]:
encrypted_message, res = cipher_DES(binary_message, binary_key)
print(f"Encrypted message: {encrypted_message.to01()}")

Encrypted message: 1000010000111111110111101110011010011111011001000001101100000111


In [21]:
decrypted_message, _ = cipher_DES(encrypted_message, binary_key, encrypt=False)
print(f"Decrypted message: {decrypted_message.to01()}")

Decrypted message: 0000000110100010101100111100010011010101111001100000011111111000


In [22]:
print(f"Encrypted message (hex): {bin_to_hex(encrypted_message)}")
print(f"Decrypted message (hex): {bin_to_hex(decrypted_message)}")

Encrypted message (hex): 84 3F DE E6 9F 64 1B 07
Decrypted message (hex): 01 A2 B3 C4 D5 E6 07 F8


In [23]:
# in ra tất cả các bước trong quá trình mã hóa
for i, (left, right, key) in enumerate(res):
    print(f"Round {i + 1}:")
    print(f"Left: {left.to01()}")
    print(f"Right: {right.to01()}")
    print(f"Key: {key.to01()}")
    print()

Round 1:
Left: 10111110101001101000000001100110
Right: 10010111011110100110001001010101
Key: 000110110000001011101111111111000111000001110010

Round 2:
Left: 10010111011110100110001001010101
Right: 00011110101011111011111110110111
Key: 011110011010111011011001110110111100100111100101

Round 3:
Left: 00011110101011111011111110110111
Right: 11110101001001101100010011000110
Key: 010101011111110010001010010000101100111110011001

Round 4:
Left: 11110101001001101100010011000110
Right: 01000111111001010101111011101110
Key: 011100101010110111010110110110110011010100011101

Round 5:
Left: 01000111111001010101111011101110
Right: 10001011101000011010001111000000
Key: 011111001110110000000111111010110101001110101000

Round 6:
Left: 10001011101000011010001111000000
Right: 01010001001000101101110111110111
Key: 011000111010010100111110010100000111101100101111

Round 7:
Left: 01010001001000101101110111110111
Right: 00100010000100001000110001011000
Key: 111011001000010010110111111101100001100010111100


In [24]:
# in kết quả ra file excel gồm các cột: Round, Left, Right, Key
import pandas as pd
res = [(left.to01(), right.to01(), key.to01()) for left, right, key in res]
df = pd.DataFrame(res, columns=["Left", "Right", "Key"])
df.to_excel("DES_rounds.xlsx")

# **2. Modes DES (ECB, CBC)**

In [25]:
"""Viết hàm chuyển chuỗi thành dạng nhị phân dưới dạng các block 64 bit"""
def str_to_block_64_bit(string):
    binary_string = bitarray()
    binary_string.frombytes(string.encode("ISO-8859-1"))
    padding = 64 - len(binary_string) % 64
    binary_string += bitarray(padding * "0")
    return [binary_string[i:i+64] for i in range(0, len(binary_string), 64)]

In [26]:
def block_64_bit_to_str(blocks):
    binary_string = bitarray()
    for block in blocks:
        binary_string += block
    return binary_string.tobytes().decode("ISO-8859-1")

In [27]:
# DES в режимах ECB
def DES_mode_ECB(message, key, encrypt=True):
    blocks = str_to_block_64_bit(message)
    res_blocks = []
    for block in blocks:
        res_block, _ = cipher_DES(block, key, encrypt)
        res_blocks.append(res_block)
    
    result_hex = ' '.join([bin_to_hex(block) for block in res_blocks])
    result_str = block_64_bit_to_str(res_blocks)
    return result_hex, result_str

In [28]:
message = "Hello, world! My name is DES. DES is a symmetric-key algorithm for the encryption of electronic data."
blocks = str_to_block_64_bit(message)
encrypted_message_hex_ECB, encrypted_message_str_ECB = DES_mode_ECB(message, binary_key, encrypt=True)
decrypted_message_hex_ECB, decrypted_message_str_ECB = DES_mode_ECB(encrypted_message_str_ECB, binary_key, encrypt=False)

In [29]:
print(f"Encrypted message by mode ECB (hex): {encrypted_message_hex_ECB}")
print(f"Encrypted message by mode ECB (str): {encrypted_message_str_ECB}")  
print(f"Decrypted message by mode ECB (hex): {decrypted_message_hex_ECB}")
print(f"Decrypted message by mode ECB (str): {decrypted_message_str_ECB}")

Encrypted message by mode ECB (hex): 73 22 23 38 BD 44 D3 05 4A D0 AE 85 BF F1 4D 57 24 4D AF 61 9C 4B 09 EF 2C 27 9A A2 E0 4C CF 05 24 A5 B3 2F 1D E8 3F 01 65 FB BD 0A F4 32 22 7E D7 05 87 C9 9E 17 CE 78 C0 42 39 1E 0D 6A E7 5A 64 9F 48 D2 E0 44 73 14 B7 7B 01 22 9E 17 87 03 F6 75 03 E7 45 AF 59 D8 DC 6E D9 21 25 F4 B1 03 DE C0 8C CE D2 79 20 70
Encrypted message by mode ECB (str): s"#8½DÓJÐ®¿ñMW$M¯aK	ï,'¢àLÏ$¥³/è?eû½
jçZdHÒàDs·{"öuçE¯YØÜnÙ!%ô±ÞÀÎÒy p
Decrypted message by mode ECB (hex): 48 65 6C 6C 6F 2C 20 77 6F 72 6C 64 21 20 4D 79 20 6E 61 6D 65 20 69 73 20 44 45 53 2E 20 44 45 53 20 69 73 20 61 20 73 79 6D 6D 65 74 72 69 63 2D 6B 65 79 20 61 6C 67 6F 72 69 74 68 6D 20 66 6F 72 20 74 68 65 20 65 6E 63 72 79 70 74 69 6F 6E 20 6F 66 20 65 6C 65 63 74 72 6F 6E 69 63 20 64 61 74 61 2E 00 00 00 9E FD FC 5C 2B 5C D5 85
Decrypted message by mode ECB (str): Hello, world! My name is DES. DES is a symmetric-key algorithm for the encryption of electronic data.   ýü\+\Õ


In [30]:
#  DES в режимах CBC
def DES_mode_CBC(message, key, iv, encrypt=True):
    blocks = str_to_block_64_bit(message)
    res_blocks = []
    previous_block = iv
    for block in blocks:
        """ Encrypt """
        block = block ^ previous_block if encrypt else block
        res_block, _ = cipher_DES(block, key, encrypt)
        """ Decrypt """
        res_block = res_block ^ previous_block if not encrypt else res_block
        res_blocks.append(res_block)
    
    result_hex = ' '.join([bin_to_hex(block) for block in res_blocks])
    result_str = block_64_bit_to_str(res_blocks)
    return result_hex, result_str


In [31]:
"""random iv 64 bit"""
import random
iv = bitarray()
for _ in range(64):
    iv.append(random.choice([0, 1]))

encrypted_message_hex_CBC, encrypted_message_str_CBC = DES_mode_CBC(message, binary_key, iv, encrypt=True)
decrypted_message_hex_CBC, decrypted_message_str_CBC = DES_mode_CBC(encrypted_message_str_CBC, binary_key, iv, encrypt=False)


In [32]:
print(f"Encrypted message by mode CBC (hex): {encrypted_message_hex_CBC}")
print(f"Encrypted message by mode CBC (str): {encrypted_message_str_CBC}")
print(f"Decrypted message by mode CBC (hex): {decrypted_message_hex_CBC}")
print(f"Decrypted message by mode CBC (str): {decrypted_message_str_CBC}")

Encrypted message by mode CBC (hex): 6F 0C 3C 02 44 5D 62 6B 75 19 99 21 E6 51 EA 83 EF 0D EE D6 9A 93 11 76 15 EC D6 18 B8 23 0C 7E 24 F6 5A 43 4D 7F 5C E5 0C CB 76 9A E7 B7 4C 2D DB 86 9D 7F CC AD 1D 86 5B D3 02 CC 0E 94 61 12 CE 55 D4 BF 83 4B 29 9E 4A 90 1B 66 2C 1C 67 4A 31 20 F6 51 F4 27 59 34 91 8D 97 49 89 6F 39 90 5D 35 C6 60 1C 3C D7 5B
îÖvìÖ¸#~$öZCM\åËvç·L-ÛÌ­[ÓÌaÎUÔ¿K)Jf,gJ1 öQô'Y4Io9]5Æ`<×[
Decrypted message by mode CBC (hex): 48 65 6C 6C 6F 2C 20 77 6F 72 6C 64 21 20 4D 79 20 6E 61 6D 65 20 69 73 20 44 45 53 2E 20 44 45 53 20 69 73 20 61 20 73 79 6D 6D 65 74 72 69 63 2D 6B 65 79 20 61 6C 67 6F 72 69 74 68 6D 20 66 6F 72 20 74 68 65 20 65 6E 63 72 79 70 74 69 6F 6E 20 6F 66 20 65 6C 65 63 74 72 6F 6E 69 63 20 64 61 74 61 2E 00 00 00 4D 18 C4 E5 D5 01 12 E5
Decrypted message by mode CBC (str): Hello, world! My name is DES. DES is a symmetric-key algorithm for the encryption of electronic data.   MÄåÕå


In [33]:
# in ra tất cả kết quả của hai chế độ ECB và CBC ra file text
with open("DES_modes_results.txt", "w", encoding="utf-8") as file:
    file.write(f"Message: {message}\n")
    file.write(f"Key: {bin_to_hex(binary_key)}\n")
    file.write(f"IV: {bin_to_hex(iv)}\n")

    file.write("\nDES mode ECB:\n")
    file.write("Encrypted message by mode ECB (hex):\n")
    for i in range(0, len(encrypted_message_hex_ECB), 63):
        file.write(encrypted_message_hex_ECB[i:i+63] + "\n")
    file.write(f"\nEncrypted message by mode ECB (str): {encrypted_message_str_ECB}\n")
    
    file.write("\n\nDecrypted message by mode ECB (hex):\n")
    for i in range(0, len(decrypted_message_hex_ECB), 63):
        file.write(decrypted_message_hex_ECB[i:i+63] + "\n")
    file.write(f"\nDecrypted message by mode ECB (str): {decrypted_message_str_ECB}\n")
    
    file.write("\nDES mode CBC:\n")
    file.write("Encrypted message by mode CBC (hex):\n")
    for i in range(0, len(encrypted_message_hex_CBC), 63):
        file.write(encrypted_message_hex_CBC[i:i+63] + "\n")
    file.write(f"\nEncrypted message by mode CBC (str): {encrypted_message_str_CBC}\n")
    
    file.write("\nEncrypted message by mode CBC (hex):\n")
    for i in range(0, len(decrypted_message_hex_CBC), 63):
        file.write(decrypted_message_hex_CBC[i:i+63] + "\n")
    file.write(f"\nDecrypted message by mode CBC (str): {decrypted_message_str_CBC}\n")

# **3. 3-DES (Triple Data Encryption Standard)**

In [34]:
message = "Hello, world! My name is 3-DES. 3-DES is a symmetric-key algorithm for the encryption of electronic data."
key1 = "13 34 57 79 9B BC DF F1"
key2 = "1F 32 45 67 89 AB CD EF"
key3 = "2B 4D 6F 81 A3 C5 E7 F9"

In [35]:
def DES_EDE3(message, key1, key2, key3, encrypt=True):
    # kiểm tra key1 nếu là hex thì chuyển thành binary
    if isinstance(key1, str):
        key1 = hex_to_bin(key1)
    if isinstance(key2, str):
        key2 = hex_to_bin(key2)
    if isinstance(key3, str):
        key3 = hex_to_bin(key3)
    result = ''
    if encrypt:
        result, _ = cipher_DES(message, key1)
        result, _ = cipher_DES(result, key2, encrypt=False)
        result, _ = cipher_DES(result, key3)
    else:
        result, _ = cipher_DES(message, key3, encrypt=False)
        result, _ = cipher_DES(result, key2)
        result, _ = cipher_DES(result, key1, encrypt=False)
    return result

In [36]:
def DES_EDE3_mode_ECB(message, key1, key2, key3, encrypt=True):
    blocks = str_to_block_64_bit(message)
    res_blocks = []
    for block in blocks:
        res_block = DES_EDE3(block, key1, key2, key3, encrypt)
        res_blocks.append(res_block)
    
    result_hex = ' '.join([bin_to_hex(block) for block in res_blocks])
    result_str = block_64_bit_to_str(res_blocks)
    return result_hex, result_str

In [37]:
encrypted_message_hex_EDE3_ECB, encrypted_message_str_EDE3_ECB = DES_EDE3_mode_ECB(message, key1, key2, key3, encrypt=True)
decrypted_message_hex_EDE3_ECB, decrypted_message_str_EDE3_ECB = DES_EDE3_mode_ECB(encrypted_message_str_EDE3_ECB, key1, key2, key3, encrypt=False)

In [38]:
print(f"Encrypted message by 3-DES (hex): {encrypted_message_hex_EDE3_ECB}")
print(f"Encrypted message by 3-DES (str): {encrypted_message_str_EDE3_ECB}")
print(f"Decrypted message by 3-DES (hex): {decrypted_message_hex_EDE3_ECB}")
print(f"Decrypted message by 3-DES (str): {decrypted_message_str_EDE3_ECB}")

Encrypted message by 3-DES (hex): 29 68 10 C9 DF FD E0 78 A3 D6 0E 50 9A B0 70 B9 31 88 F1 4F 23 10 19 93 85 BC F7 E5 DE 89 15 E5 FB DF 6E 4A C6 D4 06 87 DA EF 90 51 1B 09 BC 3E DB E4 3E BA DB 12 9B CA B0 17 1A 0F D7 A3 82 0E DB 42 3A C3 89 0F 61 A7 82 CF BB ED 3B 5E F4 7A 30 61 96 6A C4 15 A5 77 3A E8 CA 01 2B 94 C1 93 45 24 89 D1 18 9C 5E 23 8B 79 79 78 DB F7 8B 6A
Encrypted message by 3-DES (str): )hÉßýàx£ÖP°p¹1ñO#¼÷åÞåûßnJÆÔÚïQ	¼>Ûä>ºÛÊ°×£ÛB:Ãa§Ï»í;^ôz0ajÄ¥w:èÊ+ÁE$Ñ^#yyxÛ÷j
Decrypted message by 3-DES (hex): 48 65 6C 6C 6F 2C 20 77 6F 72 6C 64 21 20 4D 79 20 6E 61 6D 65 20 69 73 20 33 2D 44 45 53 2E 20 33 2D 44 45 53 20 69 73 20 61 20 73 79 6D 6D 65 74 72 69 63 2D 6B 65 79 20 61 6C 67 6F 72 69 74 68 6D 20 66 6F 72 20 74 68 65 20 65 6E 63 72 79 70 74 69 6F 6E 20 6F 66 20 65 6C 65 63 74 72 6F 6E 69 63 20 64 61 74 61 2E 00 00 00 00 00 00 00 20 2E 58 7C 8A CE 46 C0
Decrypted message by 3-DES (str): Hello, world! My name is 3-DES. 3-DES is a symmetric-ke

In [39]:
def DES_EDE2(message, key1, key2, encrypt=True):
    if isinstance(key1, str):
        key1 = hex_to_bin(key1)
    if isinstance(key2, str):
        key2 = hex_to_bin(key2)
    result = ''
    if encrypt:
        result, _ = cipher_DES(message, key1)
        result, _ = cipher_DES(result, key2, encrypt=False)
        result, _ = cipher_DES(result, key1)
    else:
        result, _ = cipher_DES(message, key1, encrypt=False)
        result, _ = cipher_DES(result, key2)
        result, _ = cipher_DES(result, key1, encrypt=False)
    return result

In [40]:
def DES_EDE2_mode_ECB(message, key1, key2, encrypt=True):
    blocks = str_to_block_64_bit(message)
    res_blocks = []
    for block in blocks:
        res_block = DES_EDE2(block, key1, key2, encrypt)
        res_blocks.append(res_block)
    
    result_hex = ' '.join([bin_to_hex(block) for block in res_blocks])
    result_str = block_64_bit_to_str(res_blocks)
    return result_hex, result_str

In [41]:
encrypted_message_hex_EDE2_ECB, encrypted_message_str_EDE2_ECB = DES_EDE2_mode_ECB(message, key1, key2, encrypt=True)
decrypted_message_hex_EDE2_ECB, decrypted_message_str_EDE2_ECB = DES_EDE2_mode_ECB(encrypted_message_str_EDE2_ECB, key1, key2, encrypt=False)

In [42]:
print(f"Encrypted message by 2-DES (hex): {encrypted_message_hex_EDE2_ECB}")
print(f"Encrypted message by 2-DES (str): {encrypted_message_str_EDE2_ECB}")
print(f"Decrypted message by 2-DES (hex): {decrypted_message_hex_EDE2_ECB}")
print(f"Decrypted message by 2-DES (str): {decrypted_message_str_EDE2_ECB}")

Encrypted message by 2-DES (hex): E9 1B 70 36 E0 48 F5 02 85 AE 1D 6E DD AC 7F EA 9C 43 13 C6 C5 8E B3 8B C8 95 2A 97 A2 EB AE 3A 53 4C D9 FE 66 59 34 E9 5C 14 0F 8E 1B 5B D9 89 FA 73 3D 17 C4 CF B8 B2 7E C6 81 73 7E 8A BB 13 48 E6 E9 7B DD B1 9A 02 F9 05 83 6D EA DF 4F DA 20 92 1A ED F7 FF 15 5F FA 4F A1 C6 38 EA D8 6E 94 0F D9 92 F0 D1 EE 18 A0 0A 0E A5 58 E8 AA 6D
Encrypted message by 2-DES (str): ép6àHõ®nÝ¬êCÆÅ³È*¢ë®:SLÙþfY4é\[Ùús=ÄÏ¸²~Æs~»Hæé{Ý±ùmêßOÚ í÷ÿ_úO¡Æ8êØnÙðÑî 
¥Xèªm
Decrypted message by 2-DES (hex): 48 65 6C 6C 6F 2C 20 77 6F 72 6C 64 21 20 4D 79 20 6E 61 6D 65 20 69 73 20 33 2D 44 45 53 2E 20 33 2D 44 45 53 20 69 73 20 61 20 73 79 6D 6D 65 74 72 69 63 2D 6B 65 79 20 61 6C 67 6F 72 69 74 68 6D 20 66 6F 72 20 74 68 65 20 65 6E 63 72 79 70 74 69 6F 6E 20 6F 66 20 65 6C 65 63 74 72 6F 6E 69 63 20 64 61 74 61 2E 00 00 00 00 00 00 00 88 E6 F1 FE 40 CA 17 E7
Decrypted message by 2-DES (str): Hello, world! My name is 3-DES. 3-DES is a symmetric-ke

In [43]:
def DES_EEE3(message, key1, key2, key3, encrypt=True):
    if isinstance(key1, str):
        key1 = hex_to_bin(key1)
    if isinstance(key2, str):
        key2 = hex_to_bin(key2)
    if isinstance(key3, str):
        key3 = hex_to_bin(key3)
    result = ''
    if encrypt:
        result, _ = cipher_DES(message, key1)
        result, _ = cipher_DES(result, key2)
        result, _ = cipher_DES(result, key3)
    else:
        result, _ = cipher_DES(message, key3, encrypt=False)
        result, _ = cipher_DES(result, key2, encrypt=False)
        result, _ = cipher_DES(result, key1, encrypt=False)
    return result

In [44]:
def DES_EEE3_mode_ECB(message, key1, key2, key3, encrypt=True):
    blocks = str_to_block_64_bit(message)
    res_blocks = []
    for block in blocks:
        res_block = DES_EEE3(block, key1, key2, key3, encrypt)
        res_blocks.append(res_block)
    
    result_hex = ' '.join([bin_to_hex(block) for block in res_blocks])
    result_str = block_64_bit_to_str(res_blocks)
    return result_hex, result_str

In [45]:
encrypted_message_hex_EEE3_ECB, encrypted_message_str_EEE3_ECB = DES_EEE3_mode_ECB(message, key1, key2, key3, encrypt=True)
decrypted_message_hex_EEE3_ECB, decrypted_message_str_EEE3_ECB = DES_EEE3_mode_ECB(encrypted_message_str_EEE3_ECB, key1, key2, key3, encrypt=False)

In [46]:
print(f"Encrypted message by 3-DES (EEE) (hex): {encrypted_message_hex_EEE3_ECB}")
print(f"Encrypted message by 3-DES (EEE) (str): {encrypted_message_str_EEE3_ECB}")
print(f"Decrypted message by 3-DES (EEE) (hex): {decrypted_message_hex_EEE3_ECB}")
print(f"Decrypted message by 3-DES (EEE) (str): {decrypted_message_str_EEE3_ECB}")

Encrypted message by 3-DES (EEE) (hex): 58 64 6D 9C 1F 3A 24 3C C8 19 75 F2 B7 EF 86 95 4D BE 6F 62 3B B8 D5 70 B9 BF 76 26 2E 07 66 12 1F 44 B1 69 F9 4E C7 93 A7 55 E3 89 96 3C CB A6 20 34 1A 7F C1 9B CA 0A 1F 9A 77 35 4E 4A 94 42 22 8E 29 83 C3 BC D8 F3 59 36 B3 CA DF A3 57 65 12 95 6D 4F 67 DF 7F 91 50 18 2D 48 50 F6 FD 33 3B 16 FF A5 74 AB F3 D7 7F 8B DA CC 63 EE 7E 79
Encrypted message by 3-DES (EEE) (str): Xdm:$<Èuò·ïM¾ob;¸Õp¹¿v&.fD±iùNÇ§Uã<Ë¦ 4ÁÊ
w5NJB")Ã¼ØóY6³Êß£WemOgßP-HPöý3;ÿ¥t«ó×ÚÌcî~y
Decrypted message by 3-DES (EEE) (hex): 48 65 6C 6C 6F 2C 20 77 6F 72 6C 64 21 20 4D 79 20 6E 61 6D 65 20 69 73 20 33 2D 44 45 53 2E 20 33 2D 44 45 53 20 69 73 20 61 20 73 79 6D 6D 65 74 72 69 63 2D 6B 65 79 20 61 6C 67 6F 72 69 74 68 6D 20 66 6F 72 20 74 68 65 20 65 6E 63 72 79 70 74 69 6F 6E 20 6F 66 20 65 6C 65 63 74 72 6F 6E 69 63 20 64 61 74 61 2E 00 00 00 00 00 00 00 51 68 C7 8F 88 CE B2 3A
Decrypted message by 3-DES (EEE) (str): Hello, world! My name is 3-DES.

In [47]:
def DES_EEE2(message, key1, key2, encrypt=True):
    if isinstance(key1, str):
        key1 = hex_to_bin(key1)
    if isinstance(key2, str):
        key2 = hex_to_bin(key2)
    result = ''
    if encrypt:
        result, _ = cipher_DES(message, key1)
        result, _ = cipher_DES(result, key2)
        result, _ = cipher_DES(result, key1)
    else:
        result, _ = cipher_DES(message, key1, encrypt=False)
        result, _ = cipher_DES(result, key2, encrypt=False)
        result, _ = cipher_DES(result, key1, encrypt=False)
    return result

In [48]:
def DES_EEE2_mode_ECB(message, key1, key2, encrypt=True):
    blocks = str_to_block_64_bit(message)
    res_blocks = []
    for block in blocks:
        res_block = DES_EEE2(block, key1, key2, encrypt)
        res_blocks.append(res_block)
    
    result_hex = ' '.join([bin_to_hex(block) for block in res_blocks])
    result_str = block_64_bit_to_str(res_blocks)
    return result_hex, result_str

In [49]:
encrypted_message_hex_EEE2_ECB, encrypted_message_str_EEE2_ECB = DES_EEE2_mode_ECB(message, key1, key2, encrypt=True)
decrypted_message_hex_EEE2_ECB, decrypted_message_str_EEE2_ECB = DES_EEE2_mode_ECB(encrypted_message_str_EEE2_ECB, key1, key2, encrypt=False)

In [50]:
print(f"Encrypted message by 2-DES (EEE) (hex): {encrypted_message_hex_EEE2_ECB}")
print(f"Encrypted message by 2-DES (EEE) (str): {encrypted_message_str_EEE2_ECB}")
print(f"Decrypted message by 2-DES (EEE) (hex): {decrypted_message_hex_EEE2_ECB}")
print(f"Decrypted message by 2-DES (EEE) (str): {decrypted_message_str_EEE2_ECB}")

Encrypted message by 2-DES (EEE) (hex): 72 FD C7 CF CF 13 06 DA C0 4D EB 2A DD D5 52 06 7A 72 F2 5B C2 58 3C A8 61 EE 91 DC 6A 9A 73 5C C8 1A 1D DA BB 70 29 B9 45 08 25 D0 1C 37 72 59 78 14 41 39 82 30 75 49 A0 25 41 73 4A BA 08 FD A8 E2 B6 CD B1 D5 E4 C1 93 6E 86 9B 60 C4 35 05 02 BA ED 7C 28 3F 00 40 8E 4E A3 56 DC 0B 46 CA B3 0D 18 FF 04 75 BF E4 20 EE 91 61 9E E6 04 70
ÿu¿ä îaæpage by 2-DES (EEE) (str): rýÇÏÏÚÀMë*ÝÕRzrò[ÂX<¨aîÜjs\ÈÚ»p)¹%Ð7rYxA90uI %AsJý¨â¶Í±ÕäÁn`Ä5ºí|(? @N£VÜFÊ³
Decrypted message by 2-DES (EEE) (hex): 48 65 6C 6C 6F 2C 20 77 6F 72 6C 64 21 20 4D 79 20 6E 61 6D 65 20 69 73 20 33 2D 44 45 53 2E 20 33 2D 44 45 53 20 69 73 20 61 20 73 79 6D 6D 65 74 72 69 63 2D 6B 65 79 20 61 6C 67 6F 72 69 74 68 6D 20 66 6F 72 20 74 68 65 20 65 6E 63 72 79 70 74 69 6F 6E 20 6F 66 20 65 6C 65 63 74 72 6F 6E 69 63 20 64 61 74 61 2E 00 00 00 00 00 00 00 83 CC D6 94 7D 08 B8 94
Decrypted message by 2-DES (EEE) (str): Hello, world! My name is 3-DES. 3-DES is a symmetr